In [209]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# Instantiate model

In [210]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Encode and Calculate

In [211]:
tokens = tokenizer.encode("It was good butcould be better.", return_tensors = "pt")

In [212]:
result = model(tokens)

In [213]:
result.logits

tensor([[-1.6899,  0.2065,  1.9689,  1.0088, -1.2988]],
       grad_fn=<AddmmBackward>)

In [214]:
int(torch.argmax(result.logits)) + 1

3

# Collect Review

In [215]:
r = requests.get("https://www.yelp.com/biz/social-brew-cafe-pyrmont")
soup = BeautifulSoup(r.text, "html.parser")
regex = re.compile(".*comment.*")
results = soup.find_all("p", {"class": regex})
reviews = [result.text for result in results] 

In [216]:
reviews[1]

'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer.'

## Load Reviewsinto DataFrame and Score

In [217]:
import numpy as np
import pandas as pd

In [218]:
df = pd.DataFrame(np.array(reviews), columns = ["review"])

In [219]:
df.head()

,review
0,Still one of the favorite coffee shop in Sydne...
1,I came to Social brew cafe for brunch while ex...
2,Ricotta hot cakes! These were so yummy. I ate ...
3,Good coffee and toasts. Straight up and down -...
4,Ron & Jo are on the go down under and Wow! We...


In [220]:
df["review"].iloc[0]

'Still one of the favorite coffee shop in Sydney. Staffs have excellent knowledge about beans , flavor, brew skills. They make own pastries which are also tasty too.'

In [221]:
def sentimentScore(review):
    sentiment_tokens = tokenizer.encode(review, return_tensors = "pt")
    result = model(sentiment_tokens)
    return int(torch.argmax(result.logits)) + 1

In [222]:
sentimentScore(df["review"].iloc[1])

5

In [223]:
df["sentiment"] = df["review"].apply(lambda x: sentimentScore(x[:512]))

In [224]:
df

,review,sentiment
0,Still one of the favorite coffee shop in Sydne...,5
1,I came to Social brew cafe for brunch while ex...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5
3,Good coffee and toasts. Straight up and down -...,5
4,Ron & Jo are on the go down under and Wow! We...,5
5,Great coffee and vibe. That's all you need. C...,5
6,Great coffee and vibe. That's all you need. C...,4
7,We came for brunch twice in our week-long visi...,4
8,This is my fave brunch café in and around Sydn...,5
9,Delicious. The waitress was hot. The burger wa...,4
